# 3D Volumetric Annotation with OMERO-Annotate-AI

This notebook demonstrates how to use the 3D volumetric annotation features in omero-annotate-ai.

## Features Covered:
- 3D volumetric processing vs 2D slice-by-slice
- Configuration for 3D workflows
- Table schema with 3D fields
- Volumetric ROI creation

In [ ]:
import os
from pathlib import Path
from omero_annotate_ai import create_omero_connection_widget, create_workflow_widget, create_pipeline
from omero_annotate_ai.core.annotation_config import AnnotationConfig, SpatialCoverage

## Step 1: Connect to OMERO

First, establish a connection to your OMERO server:

In [ ]:
# Create and display connection widget
conn_widget = create_omero_connection_widget()
conn_widget.display()

# Get connection after user authentication
conn = conn_widget.get_connection()

## Step 2: Configure 3D Volumetric Workflow

Configure the workflow for true 3D volumetric processing:

In [ ]:
# Option 1: Use workflow widget for interactive configuration
workflow_widget = create_workflow_widget(connection=conn)
workflow_widget.display()

# Get configuration from widget
config = workflow_widget.get_config()

In [ ]:
# Option 2: Create 3D configuration programmatically
config_3d = AnnotationConfig(
    name="3d_volumetric_annotation",
    spatial_coverage=SpatialCoverage(
        channels=[0],  # DAPI channel
        timepoints=[0],  # Single timepoint
        z_slices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],  # 10 z-slices
        z_slice_mode="specific",
        three_d=True,  # Enable 3D volumetric processing
        z_range_start=0,  # Start of volume
        z_range_end=9,    # End of volume
    ),
    # Configure for your OMERO container
    omero={
        "container_type": "dataset",
        "container_id": 123,  # Replace with your dataset ID
        "source_desc": "3D nuclei segmentation dataset"
    },
    # Processing settings
    processing={
        "batch_size": 1,  # Process one volume at a time
        "use_patches": False,  # Full volume processing
    },
    # AI model settings
    ai_model={
        "name": "micro-sam",
        "model_type": "vit_b_lm",  # Suitable for 3D
    },
    # Training settings
    training={
        "train_n": 2,
        "validate_n": 1,
        "segment_all": False,
    }
)

print(f"✅ 3D Configuration created:")
print(f"  - Volumetric processing: {config_3d.spatial_coverage.is_volumetric}")
print(f"  - Z-range: {config_3d.spatial_coverage.get_z_range()}")
print(f"  - Z-length: {config_3d.spatial_coverage.get_z_length()}")

## Step 3: Create and Run 3D Pipeline

Create the annotation pipeline with 3D support:

In [ ]:
# Create pipeline with 3D configuration
pipeline = create_pipeline(config_3d, conn)

# Run the full workflow
print("🚀 Starting 3D volumetric annotation workflow...")
table_id, processed_images = pipeline.run_full_workflow()

print(f"\n✅ 3D annotation workflow completed!")
print(f"📋 Table ID: {table_id}")
print(f"📊 Processed {len(processed_images)} images as 3D volumes")

## Step 4: Verify 3D Table Schema

Check that the annotation table includes the new 3D fields:

In [ ]:
import ezomero
import pandas as pd

# Load the annotation table
df = ezomero.get_table(conn, table_id)

print(f"📋 Table Schema ({len(df)} rows):")
print(f"   Columns: {list(df.columns)}")

# Check for 3D-specific fields
has_3d_fields = all(col in df.columns for col in ['z_start', 'z_end', 'z_length'])
print(f"\n✅ 3D fields present: {has_3d_fields}")

if has_3d_fields:
    volumetric_rows = df[df['is_volumetric'] == True]
    print(f"📊 Volumetric entries: {len(volumetric_rows)}")
    
    if len(volumetric_rows) > 0:
        print("\n🔍 Sample 3D entry:")
        sample = volumetric_rows.iloc[0]
        print(f"   Image ID: {sample['image_id']}")
        print(f"   Z-range: {sample['z_start']} - {sample['z_end']}")
        print(f"   Z-length: {sample['z_length']}")
        print(f"   Is volumetric: {sample['is_volumetric']}")
        print(f"   Processed: {sample['processed']}")

## Step 5: Compare 2D vs 3D Processing

Demonstrate the difference between 2D slice-by-slice and 3D volumetric processing:

In [ ]:
# Create equivalent 2D configuration for comparison
config_2d = AnnotationConfig(
    name="2d_slice_annotation",
    spatial_coverage=SpatialCoverage(
        channels=[0],
        timepoints=[0],
        z_slices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],  # Same z-slices
        z_slice_mode="specific",
        three_d=False,  # 2D slice-by-slice processing
    ),
    omero={"container_type": "dataset", "container_id": 123},
    processing={"batch_size": 1, "use_patches": False},
    ai_model={"name": "micro-sam", "model_type": "vit_b_lm"},
    training={"train_n": 1, "validate_n": 1, "segment_all": False}
)

print("📊 Processing Mode Comparison:")
print(f"\n3D Volumetric Mode:")
print(f"  - Volume processing: {config_3d.spatial_coverage.is_volumetric}")
print(f"  - Expected table rows per image: 1 (full volume)")
print(f"  - Z-range per row: {config_3d.spatial_coverage.get_z_range()}")

print(f"\n2D Slice Mode:")
print(f"  - Volume processing: {config_2d.spatial_coverage.is_volumetric}")
print(f"  - Expected table rows per image: {len(config_2d.spatial_coverage.z_slices)} (one per slice)")
print(f"  - Z-range per row: Single slice")

## Step 6: Configuration Management

Save and load 3D configurations:

In [ ]:
# Save 3D configuration to YAML
config_path = Path("3d_volumetric_config.yaml")
config_3d.save_yaml(config_path)
print(f"💾 3D configuration saved to: {config_path}")

# Load configuration from YAML
loaded_config = AnnotationConfig.from_yaml(config_path)
print(f"📂 Configuration loaded successfully")
print(f"   Name: {loaded_config.name}")
print(f"   Volumetric: {loaded_config.spatial_coverage.is_volumetric}")

# Display YAML content
print(f"\n📄 Configuration YAML:")
with open(config_path, 'r') as f:
    print(f.read())

## Key 3D Features Summary

### Configuration Changes:
- **`three_d: true`** - Enables 3D volumetric processing mode
- **`z_range_start/z_range_end`** - Defines volume extent
- **Backward compatibility** - Existing 2D configs work unchanged

### Table Schema Enhancements:
- **`z_start`** - Starting z-slice of volume
- **`z_end`** - Ending z-slice of volume  
- **`z_length`** - Number of z-slices in volume
- **`is_volumetric`** - Boolean flag for volume processing

### Processing Differences:
- **2D Mode**: One table row per z-slice
- **3D Mode**: One table row per volume (all z-slices)
- **ROIs**: 3D volumes create multi-slice ROIs

### Benefits:
- **Reduced table size** - Fewer rows for 3D data
- **Volume-aware processing** - Better for 3D structures
- **Flexible z-ranges** - Process subsets of z-stacks
- **Training compatibility** - Works with existing micro-SAM models